In [1]:
import random
import os

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torchvision
from torchvision.transforms import v2

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm

import matplotlib.pyplot as plt

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda')

In [2]:
data_path = "./data/count_animals/data/data_for_achive/train"
data = pd.read_csv(os.path.join(data_path, "answers.csv"))
data = data.rename(columns={"Unnamed: 0": "picture_path"})
data

,picture_path,count
0,train_0000.png,2
1,train_0001.png,3
2,train_0002.png,4
3,train_0003.png,1
4,train_0004.png,4
...,...,...
9995,train_9995.png,3
9996,train_9996.png,2
9997,train_9997.png,2
9998,train_9998.png,3


In [3]:
y = data["count"].tolist()
X = data["picture_path"].tolist()

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

In [5]:
class AnimalCountDataset(Dataset):
    def __init__(self, X, y, dir_path, transform):
        self.X = X
        self.y = y
        self.dir_path = dir_path
        self.transform = transform

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, indx):
        img_path = os.path.join(self.dir_path, "pictures", self.X[indx])
        img = torchvision.io.read_image(img_path)[:3]
        img = self.transform(img)
        target = self.y[indx]
        return {
            "img": img,
            "target": target
        }


In [6]:
transform = v2.Compose([
    v2.Resize((96, 96)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
batch_size = 64

train_dataset = AnimalCountDataset(X_train, y_train, data_path, transform)
val_dataset = AnimalCountDataset(X_val, y_val, data_path, transform)

train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size,
                              shuffle=True)

val_dataloader = DataLoader(val_dataset, 
                            batch_size=batch_size,
                            shuffle=False)

In [8]:
class RegressionNet(nn.Module):
    def __init__(self, num_layers, input_dim, kernel_size=3, padding=0, stride=1):
        super().__init__()
        
        self.convolutions = nn.ModuleList()

        out_channels = 8
        out_dim = (input_dim - 3) + 1
        
        layer = nn.Sequential(nn.Conv2d(3, out_channels, 
                                        kernel_size=kernel_size, 
                                        padding=padding, 
                                        stride=stride),
                              nn.ReLU())
        
        out_dim = self._calculate_output_dim(input_dim, 
                                             kernel_size,
                                             padding, 
                                             stride)
        self.convolutions.append(layer)
        
        for _ in range(num_layers - 1):
            layer = nn.Sequential(nn.BatchNorm2d(out_channels),
                                  nn.Conv2d(out_channels, out_channels * 2,
                                            kernel_size=kernel_size, 
                                            padding=padding, 
                                            stride=stride),
                                  nn.ReLU())
            self.convolutions.append(layer)
            
            out_channels *= 2
            out_dim = self._calculate_output_dim(out_dim, 
                                                 kernel_size,
                                                 padding, 
                                                 stride)
        self.flatten = nn.Flatten()

        self.regressor = nn.Sequential(
            nn.Linear(in_features=out_channels * out_dim ** 2, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=1),
        )

    
    def _calculate_output_dim(self, input_dim, kernel_size=3, padding=0, stride=1):
        out_dim = (input_dim - kernel_size + 2 * padding) / stride + 1
        assert int(out_dim) == out_dim, out_dim
        return int(out_dim)
    
    def forward(self, x):
        for layer in self.convolutions:
            x = layer(x)
        x = self.flatten(x)
        out = self.regressor(x)
        return out

In [9]:
def train_epoch(model, dataloader, optimizer, loss_funtion, scheduler=None, device="cpu"):
    model.to(device)
    model.train()
    
    preds = list()
    targets = list()
    
    for batch in tqdm(dataloader):
        optimizer.zero_grad()
        
        img = batch["img"].to(device)
        target = batch["target"].to(device)

        output = model(img).squeeze()
        
        preds.extend(output.cpu().tolist())
        targets.extend(target.cpu().tolist())
    
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
    
    mae_train = mean_absolute_error(targets, preds)
    print(f"Train MAE: {mae_train}")
    return mae_train
        
def validate(model, dataloader, device="cpu"):
    model.to(device)
    
    model.eval()
    preds = list()
    targets = list()
    with torch.inference_mode():
        for batch in tqdm(dataloader):
            img = batch["img"].to(device)
            target = batch["target"].to(device)

            output = model(img).squeeze()
            
            preds.extend(output.cpu().tolist())
            targets.extend(target.cpu().tolist())
    
    mae_val = mean_absolute_error(targets, preds)
    print(f"Val MAE: {mae_val}")
    return mae_val

In [10]:
model = RegressionNet(num_layers=6, input_dim=96, kernel_size=9, padding=0, stride=1)
model.to(device)

RegressionNet(
  (convolutions): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 8, kernel_size=(9, 9), stride=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(8, 16, kernel_size=(9, 9), stride=(1, 1))
      (2): ReLU()
    )
    (2): Sequential(
      (0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(16, 32, kernel_size=(9, 9), stride=(1, 1))
      (2): ReLU()
    )
    (3): Sequential(
      (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1))
      (2): ReLU()
    )
    (4): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(64, 128, kernel_size=(9, 9), stride=(1, 1))
      (2): ReLU()
    )
    (5): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, mome

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
loss_function = torch.nn.L1Loss()

In [12]:
epoch_number = 20
best_mae = 10 ** 8
save_path = "./count_animals_models/5.pth"

for epoch in range(epoch_number):
    print(f"Epoch: {epoch + 1}")
    mae_train = train_epoch(model, train_dataloader, optimizer, loss_function, device=device)
    mae_val = validate(model, val_dataloader, device)
    if mae_val < best_mae:
        torch.save(model, save_path)
        best_mae = mae_val

Epoch: 1


100%|██████████| 125/125 [00:15<00:00,  7.97it/s]


Train MAE: 0.9597890644185245


100%|██████████| 32/32 [00:01<00:00, 21.21it/s]


Val MAE: 0.4866317743062973
Epoch: 2


...

Val MAE: 0.198090501755476
Epoch: 20


100%|██████████| 125/125 [00:12<00:00,  9.95it/s]


Train MAE: 0.17812437042593957


100%|██████████| 32/32 [00:01<00:00, 23.67it/s]

Val MAE: 0.230790002733469


### Submission

In [13]:
test_path = "./data/count_animals/data/data_for_achive/test"
test = pd.read_csv(os.path.join(test_path, "image_names.csv"))
test = test.rename(columns={"name": "picture_path"})
test

,picture_path
0,test_0000.png
1,test_0001.png
2,test_0002.png
3,test_0003.png
4,test_0004.png
...,...
4995,test_4995.png
4996,test_4996.png
4997,test_4997.png
4998,test_4998.png


In [14]:
sample_submission = pd.read_csv(os.path.join(test_path, "sample_submission.csv"))
sample_submission

,count
0,3
1,3
2,3
3,3
4,3
...,...
4995,3
4996,3
4997,3
4998,3


In [15]:
class AnimalCountTestDataset(Dataset):
    def __init__(self, X, dir_path, transform):
        self.X = X
        self.dir_path = dir_path
        self.transform = transform

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, indx):
        img_path = os.path.join(self.dir_path, "pictures", self.X[indx])
        img = torchvision.io.read_image(img_path)[:3]
        img = self.transform(img)
        return {
            "img": img,
        }


In [16]:
test_dataset = AnimalCountTestDataset(test["picture_path"].tolist(), test_path, transform)

test_dataloader = DataLoader(test_dataset, 
                             batch_size=batch_size,
                             shuffle=False)

In [17]:
model = torch.load(save_path)

In [18]:
def test(model, dataloader, device="cpu"):
    model.to(device)
    
    model.eval()
    preds = list()

    with torch.inference_mode():
        for batch in tqdm(dataloader):
            img = batch["img"].to(device)

            output = model(img).squeeze()
            
            preds.extend(output.cpu().tolist())
    
    return preds

In [19]:
preds = test(model, test_dataloader, device=device)
preds = [max(pred, 1) for pred in preds]

100%|██████████| 79/79 [00:03<00:00, 21.83it/s]


In [20]:
sample_submission["count"] = preds
sample_submission.to_csv("./data/count_animals/submissions/sub_5.csv", index=False)